In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install protobuf==3.20.3
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

In [3]:
import json
import numpy as np
import re
import pickle
import os
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

2025-11-26 17:40:26.414606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764178826.592360      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764178826.644888      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
PRODUCTS_FILE = '/kaggle/input/crawldb/thinkpro_final.json' 

In [5]:
def create_product_embedding_data(details):
    """
    Xử lý dữ liệu sản phẩm thành 2 định dạng:
    1. 'search_text': Chuỗi gộp đầy đủ ngữ cảnh (Dùng cho Semantic Search).
    2. 'parts': Các thành phần tách biệt (Dùng cho Weighted Recommendation).
    """
    # 1. XỬ LÝ DỮ LIỆU CƠ BẢN
    name = details.get('name', '').strip()
    brand = details.get('brand', '').strip()
    category = details.get('category', '').strip()
    
    # 2. XỬ LÝ CẤU HÌNH CHUNG (Common Specs)
    common_specs_list = []
    if details.get('common_specs'):
        for k, v in details['common_specs'].items():
            # Xóa xuống dòng thừa, thay bằng khoảng trắng để không dính chữ
            clean_val = str(v).replace('\n', ' ').strip()
            common_specs_list.append(f"{k} {clean_val}")
    common_specs_str = ". ".join(common_specs_list)

    # 3. XỬ LÝ BIẾN THỂ (Variants)
    variant_texts = []
    if details.get('variants'):
        for v in details['variants']:
            variant_name = v.get('name', '').strip()
            attrs = v.get('attributes')
            attr_list = []
            
            if attrs and isinstance(attrs, dict):
                for attr_key, attr_val in attrs.items():
                    if attr_val:
                        attr_list.append(f"{attr_key} {attr_val}".strip())
            
            # Logic ghép tên và thuộc tính thông minh hơn
            if variant_name:
                if attr_list:
                    # Ví dụ: "Dell XPS i5 (RAM 16GB, SSD 512GB)"
                    variant_texts.append(f"{variant_name} ({', '.join(attr_list)})")
                else:
                    variant_texts.append(variant_name)
    
    # Lọc trùng lặp (Set) để tối ưu độ dài vector
    variants_str = ". ".join(list(set(variant_texts)))

    # 4. XỬ LÝ MÔ TẢ (Description)
    desc = details.get('description_md', '')
    desc = re.sub(r'!\[.*?\]\(.*?\)', '', desc) # Bỏ ảnh
    desc = re.sub(r'\[.*?\]\(.*?\)', '', desc)  # Bỏ link markdown
    # Thay ký tự đặc biệt bằng khoảng trắng (tránh xóa mất dấu gạch ngang của mã chip i5-12400)
    desc = re.sub(r'[#*>]', ' ', desc)          
    desc = re.sub(r'\s+', ' ', desc).strip()    # Xóa khoảng trắng thừa

    # --- MỤC ĐÍCH 1: TẠO FULL TEXT CHO SEARCH ---
    # Sắp xếp theo thứ tự quan trọng để model đọc hiểu ngữ cảnh
    full_text_search = (
        f"Sản phẩm: {name}. "
        f"Thương hiệu: {brand}. "
        f"Danh mục: {category}. "
        f"Cấu hình chung: {common_specs_str}. "
        f"Các tùy chọn biến thể: {variants_str}. " 
        f"Mô tả chi tiết: {desc}"                
    ).lower()

    # --- MỤC ĐÍCH 2: TRẢ VỀ CÁC PHẦN RIÊNG CHO RECOMMENDATION ---
    specs_combined = f"{common_specs_str}. {variants_str}".lower()
    desc_have_name = f"Sản phẩm: {name} . Mô tả: {desc}" 

    return {
        "search_text": full_text_search, 
        "parts": {                       
            "brand": brand.lower(),       
            "category": category.lower(), 
            "specs": specs_combined,     
            "desc": desc_have_name.lower()         
        }
    }

In [14]:
MODEL_NAME = 'dangvantuan/vietnamese-document-embedding'
model_large = SentenceTransformer(MODEL_NAME, trust_remote_code=True)
model_large.max_seq_length = 4096

model_small = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [13]:
print(f" Model Large: {model_large.get_sentence_embedding_dimension()} dim")
print(f" Model Small: {model_small.get_sentence_embedding_dimension()} dim")

 Model Large: 768 dim
 Model Small: 384 dim


In [15]:
try:
    with open(PRODUCTS_FILE, 'r', encoding='utf-8') as f:
        products = json.load(f)
except FileNotFoundError:
    products = {}

In [17]:
# config weight
W_BRAND = 2.0  # Ưu tiên Hãng
W_CATE  = 2.0  # Ưu tiên Loại
W_SPECS = 1.5  # Ưu tiên Cấu hình
W_DESC = 1.0

In [18]:
def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

In [20]:
if products:
    list_vec_search = [] # Vector 768 chiều
    list_vec_recs = []   # Vector (384 + 384 + 768 + 384) = 1920 chiều
    
    product_skus = []
    rag_corpus = []      # Chỉ lưu text search để RAG đọc
    
    for idx, (url, details) in enumerate(tqdm(products.items())):
        spu = details.get("SPU")
        
        # 1. Tạo dữ liệu text
        processed = create_product_embedding_data(details)
        parts = processed['parts']
        
        # --- A. TẠO VECTOR SEARCH 
        v_search = model_large.encode([processed['search_text']])[0]
        
        # --- B. TẠO VECTOR RECS ---
        
        v_brand = model_small.encode([parts['brand']])[0]
        v_cate  = model_small.encode([parts['category']])[0]
        v_specs  = model_small.encode([parts['specs']])[0]
        
        v_desc = model_large.encode([parts['desc']])[0]
        
        #  Chuẩn hóa & Nhân trọng số
        v_brand = normalize(v_brand) * W_BRAND
        v_cate  = normalize(v_cate)  * W_CATE
        v_specs = normalize(v_specs) * W_SPECS
        v_desc  = normalize(v_desc)  * W_DESC
        
        # nối
        v_recs = np.concatenate([v_brand, v_cate, v_specs, v_desc])
        
        # --- LƯU VÀO LIST ---
        list_vec_search.append(v_search)
        list_vec_recs.append(v_recs)
        product_skus.append(spu)
        rag_corpus.append(processed['search_text'])


    
    arr_search = np.array(list_vec_search, dtype=np.float32)
    arr_recs = np.array(list_vec_recs, dtype=np.float32)
    
    print(f" Search Vector Shape: {arr_search.shape}")
    print(f" Recs Vector Shape:   {arr_recs.shape}")  
    
    np.save('/kaggle/working/vectors_search.npy', arr_search)
    np.save('/kaggle/working/vectors_recs.npy', arr_recs)
    
    with open('/kaggle/working/product_skus.pkl', 'wb') as f:
        pickle.dump(product_skus, f)
    with open('/kaggle/working/product_full_detail.pkl', 'wb') as f:
        pickle.dump(rag_corpus, f)

100%|██████████| 586/586 [02:01<00:00,  4.84it/s]

📊 Search Vector Shape: (586, 768)
📊 Recs Vector Shape:   (586, 1920)
✅ HOÀN TẤT TOÀN BỘ!


In [22]:
print(rag_corpus[5])

sản phẩm: lenovo legion y9000p 2025. thương hiệu: lenovo. danh mục: lenovo legion. cấu hình chung: loại cpu intel - core ultra 9 275hx, 24 / 24. tốc độ 2.1ghz , lên tới 5.4ghz. bộ nhớ đệm 36mb. card onboard intel® graphics. card rời nvidia geforce rtx 5080 12gb. dung lượng 32gb ddr5 6400mhz. hỗ trợ tối đa 64gb (có thể nâng cấp). dung lượng ssd 1tb (pcie gen4). khả năng nâng cấp hỗ trợ 2 ssd. màn hình 16inches, 2560 x 1600px, 240hz. tấm phủ nhám, không cảm ứng. thông số khác tỉ lệ 16:10, oled, 1100nits, 100 srgb. khối lượng 2.57kg. kích thước 364mm x 275mm x 21.9-26.6mm. chất liệu vỏ nhôm - nhựa. dung lượng pin 80whr. webcam hd camera với công tắc e-camera. công nghệ âm thanh nahimic audio. cổng kết nối 1 cổng usb-c 3.2 gen 2 (10gbps + dgpu dp 2.1 + 140w pd 3.0) 1 cổng thunderbolt 4 (usb4 40gbps + igpu dp 2.1), 2 cổng usb-a 3.2 gen 1 (5gbps), 1 cổng usb-a 3.2 gen 2 (10gbps), 1 x hdmi 2.1, 1 x jack tai nghe 3.5mm. wifi wi-fi 7. bluetooth 5.4. os windows 11. phiên bản ultra 9 275hx, rtx 5

In [26]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pickle

# load model
vectors = np.load('vectors_search.npy')
with open('product_skus.pkl', 'rb') as f:
    slugs = pickle.load(f)

# dùng KNN
knn = NearestNeighbors(n_neighbors=20, metric='cosine')
knn.fit(vectors)

def test_search(query):
    print(f"\n tìm kiếm: '{query}'")
    
    # Encode câu query của user
    query_vec = model_large.encode([query.lower()])
    
    distances, indices = knn.kneighbors(query_vec)
    
    for i, idx in enumerate(indices[0]):
        slug = slugs[idx]
        score = distances[0][i]
        # In ra kết quả
        print(f"   #{i+1}: {slug} (Score: {score:.4f})")




test_search("Máy tính cho doanh nhân pin trâu")

test_search("Tôi muốn laptop gaming ngoại hình hầm hố")

test_search("Tôi muốn một tay cầm chơi game hiệu năng tốt, giá cả phải chăng mà phải đẹp")


test_search("Em chào các bác, do bàn phím cũ của e nó bị hư nên cần phải thay gấp. Nói về nhu cầu thì e chỉ cần layout 85% là được, không cần hostswap hay bluetooth gì cả chỉ cần có dây thêm cổng typeC là được, do con cũ em có mà chả cần dùng Em mua về để gõ code đồ nên cần cảm giác gõ tốt xíu, không biết bác nào biết con nào ngon ko (duoi 1tr) chỉ e với, thị trường h nhiều con phím quá em không biết con nào.")

# ví dụ thực tế từ bài đăng ở voz
test_search("Em sắp tới dự định mua 1 chiếc laptop dùng, hàng 2nd. Tài chính tầm 15tr đổ lại. Nhu cầu mình làm văn phòng, 1 số phần mềm về CNTT, kỹ thuật như matlab, cad,...nhu cầu di động cao, hay mang đi hiện trường làm việc, thuyết trình nên cần trọng lượng nhẹ, máy mát tý ạ. Ko cần cảm ứng. Cấu hình thì mình mong muốn ram từ 16-32GB, ssd 512GB, pin trâu tý. ")


 tìm kiếm: 'Máy tính cho doanh nhân pin trâu'
   #1: PRD-BC11DED0 (Score: 0.5141)
   #2: PRD-4BA39FB9 (Score: 0.5305)
   #3: PRD-03B4BADB (Score: 0.5306)
   #4: PRD-E94BE479 (Score: 0.5379)
   #5: PRD-800D18AD (Score: 0.5450)
   #6: PRD-CD4B3686 (Score: 0.5606)
   #7: PRD-5A99F351 (Score: 0.5671)
   #8: PRD-E0640D3E (Score: 0.5678)
   #9: PRD-E6D19E83 (Score: 0.5721)
   #10: PRD-ECFADE02 (Score: 0.5723)
   #11: PRD-33434D56 (Score: 0.5744)
   #12: PRD-08214355 (Score: 0.5746)
   #13: PRD-9E5C3D44 (Score: 0.5755)
   #14: PRD-72EC2737 (Score: 0.5757)
   #15: PRD-C2DB5145 (Score: 0.5760)
   #16: PRD-264D825E (Score: 0.5763)
   #17: PRD-FA5B66B7 (Score: 0.5783)
   #18: PRD-7EAD30E1 (Score: 0.5785)
   #19: PRD-5F595AAB (Score: 0.5814)
   #20: PRD-EDFAD741 (Score: 0.5815)

 tìm kiếm: 'Tôi muốn laptop gaming ngoại hình hầm hố'
   #1: PRD-CE1BB567 (Score: 0.4320)
   #2: PRD-51AC82E0 (Score: 0.4431)
   #3: PRD-4B7C996E (Score: 0.4572)
   #4: PRD-7392427C (Score: 0.4604)
   #5: PRD-DC4060E0 (Sc

In [28]:
# --- BỔ SUNG PHẦN GỢI Ý ---

sku_to_idx = {sku: i for i, sku in enumerate(slugs)}

def test_recommendation(history_skus, top_k=5):
    print(f"\n Gợi ý cho người dùng đã xem: {history_skus}")
    
    target_vectors = []
    valid_history = []
    
    # 1. Lấy vector của từng sản phẩm trong lịch sử
    for sku in history_skus:
        if sku in sku_to_idx:
            idx = sku_to_idx[sku]
            target_vectors.append(vectors[idx])
            valid_history.append(sku)
    
    if not target_vectors:
        print(" Không có dữ liệu lịch sử hợp lệ để gợi ý.")
        return

    # 2. TÍNH VECTOR TRUNG BÌNH (USER PROFILE)
    # Mẹo: Dùng trọng số thời gian (Time Decay) - Cái xem sau quan trọng hơn
    # Ví dụ xem 3 cái: weights = [0.8, 0.9, 1.0]
    n = len(target_vectors)
    weights = [0.9 ** (n - 1 - i) for i in range(n)] # [..., 0.81, 0.9, 1.0]
    
    # Tính trung bình có trọng số
    user_vec = np.average(target_vectors, axis=0, weights=weights)
    
    # 3. Tìm kiếm KNN
    # Lấy dư ra (top_k + len(history)) để trừ hao những cái đã xem
    dists, indices = knn.kneighbors([user_vec], n_neighbors=top_k + len(valid_history))
    
    # 4. Hiển thị kết quả (Lọc bỏ cái đã xem)
    count = 0
    print(f"   👉 Kết quả gợi ý:")
    for i, idx in enumerate(indices[0]):
        rec_sku = slugs[idx]
        
        # Nếu sản phẩm này đã có trong lịch sử thì bỏ qua (không gợi ý lại cái vừa xem)
        if rec_sku in valid_history:
            continue
            
        score = dists[0][i]
        print(f"   #{count+1}: {rec_sku} (Score: {score:.4f})")
        
        count += 1
        if count >= top_k: break

# --- CHẠY THỬ NGHIỆM GỢI Ý ---

# Giả sử từ kết quả tìm kiếm ở trên, bạn lấy ra được vài mã SKU
# Bạn hãy thay thế các mã dưới đây bằng mã thật xuất hiện trong output tìm kiếm của bạn

print("\n" + "="*30 + "\n   DEMO GỢI Ý\n" + "="*30)

# CASE 1: Khách đang xem 1 sản phẩm (Vừa tìm được Laptop Gaming)
# (Thay 'PRD-CE1BB567' bằng 1 mã thật từ kết quả search 'hầm hố' ở trên)
test_recommendation(['PRD-227DC75B']) 

# CASE 2: Khách đã xem 1 Laptop và 1 Bàn phím
# Hệ thống sẽ cố tìm cái gì đó "lai" giữa 2 nhu cầu hoặc phụ kiện phù hợp
test_recommendation(['PRD-BC11DED0', 'PRD-03F47A6C'])

# CASE 3: Giả lập lấy random từ dữ liệu thật để test
import random
if len(slugs) > 5:
    random_history = random.sample(slugs, 5) # Lấy bừa 2 sản phẩm
    test_recommendation(random_history)


   DEMO GỢI Ý

 Gợi ý cho người dùng đã xem: ['PRD-227DC75B']
   👉 Kết quả gợi ý:
   #1: PRD-2B33337F (Score: 0.3473)
   #2: PRD-76A31B49 (Score: 0.3523)
   #3: PRD-5B209041 (Score: 0.3558)
   #4: PRD-BB533A39 (Score: 0.3698)
   #5: PRD-46E9CB36 (Score: 0.3804)

 Gợi ý cho người dùng đã xem: ['PRD-BC11DED0', 'PRD-03F47A6C']
   👉 Kết quả gợi ý:
   #1: PRD-15E98D28 (Score: 0.3220)
   #2: PRD-65EB2954 (Score: 0.3285)
   #3: PRD-49FAAB51 (Score: 0.3494)
   #4: PRD-EDFAD741 (Score: 0.3614)
   #5: PRD-BF4997D3 (Score: 0.3617)

 Gợi ý cho người dùng đã xem: ['PRD-5BFABF0D', 'PRD-EA7481EF', 'PRD-C96A6652', 'PRD-2CB74A41', 'PRD-F9A9ECAA']
   👉 Kết quả gợi ý:
   #1: PRD-60DCFED3 (Score: 0.2900)
   #2: PRD-F4F01783 (Score: 0.3262)
   #3: PRD-354DCD21 (Score: 0.3561)
   #4: PRD-791E311D (Score: 0.3561)
   #5: PRD-7A507332 (Score: 0.3633)
